In [ ]:
# from google.colab import userdata
# ssh_password=userdata.get('SSH_PASSWORD')

In [ ]:
# # Install colab_ssh
# %pip install colab_ssh --upgrade

In [ ]:
# from colab_ssh import launch_ssh_cloudflared, init_git_cloudflared
# launch_ssh_cloudflared(password=ssh_password)

In [ ]:
import os
from google.colab import userdata

GITHUB_USERNAME = "MUKAMAFrancois"
REPO_NAME       = "tecGrwTechnical"
BRANCH          = "main"
GITHUB_TOKEN    = userdata.get('GITHUB_TOKEN')

# Construct URL with Token
repo_url = f"https://{GITHUB_TOKEN}@github.com/{GITHUB_USERNAME}/{REPO_NAME}.git"
project_dir = f"/content/{REPO_NAME}"

if os.path.exists(project_dir):
    print(f"Updating existing repo in {project_dir}...")
    %cd {project_dir}
    !git pull origin {BRANCH}
else:
    print(f"Cloning repo to {project_dir}...")
    %cd /content
    !git clone {repo_url}
    %cd {project_dir}

In [ ]:
%pip install -q -r requirements.txt

In [ ]:
%pip install huggingface-hub typer

In [ ]:
os.chdir("/content/{REPO_NAME}")
!ls -F

In [ ]:
!python3 hf_login.py

In [ ]:
!python3 loader.py